# 快速开始 ASE｜氮气分子在铜表面的吸附

<a href="https://bohrium.dp.tech/notebook/3862b7ca3ab04bddb43ade771e6c1d87" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:1rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：<a href="mailto:hh@shao.ac.cn"><b>阙浩辉 📨 </b></a></i><br/>
        <i>日期：2023-05-09</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i>
    </p>
</div>

🎯 <b style='color:purple'>本教程旨在<u>快速掌握</u> 使用 ASE 模块建立分子模型并进行计算。</b>

* 一键运行，你可以快速在实践中检验你的想法。

* 丰富完善的注释，对于入门者友好。

**在 [Bohrium Notebook](https://bohrium-doc.dp.tech/docs/userguide/Notebook) 界面，你可以点击界面上方蓝色按钮 `开始连接`，选择 `bohrium-notebook` 镜像及任何一款节点配置，稍等片刻即可运行。**

<div style="width:auto; height:2px; background:linear-gradient(244deg,rgba(0,0,0,0) 0%,rgba(0,0,0,0.5) 50%,rgba(0,0,0,1) 100%)"></div>

## 目标

> **使用 ASE 计算氮分子在铜表面上的吸附能并进行可视化**

在学习本教程后，你将能够：

- 使用 ASE 定义分子或晶体。
- 计算体系能量并进行结构弛豫。
- 使用 ASE 读取或写入原子文件
- 可视化原子文件
- 使用 ASE 进行分子动力学计算

**阅读该教程【最多】约需 10 分钟，让我们开始吧！**


## 目录

<div align="left" style="margin:1.5rem"><img src="https://gitlab.com/uploads/-/system/project/avatar/470007/ase256.png?width=64" alt="pandas" style="zoom: 200%;"></div>

* [背景](#background)
* [实践](#practice)
  * [1 案例：氮气分子在铜表面的吸附](#case)
    * [1.1 原子](#1-1)
    * [1.2 添加计算](#1-2)
    * [1.3 结构弛豫](#1-3)
    * [1.4 总结](#1-4)
  * [2 输入与输出](#io)
  * [3 可视化](#visualize)
  * [4 运行分子动力学](#molecular-dynamics)
* [总结](#summary)
* [参考资料](#references)

## 实践 <a id='practice'></a>

### 1 案例：氮气分子在铜表面的吸附 <a id='case'></a>

The Atomic Simulation Environment（ASE）是一个用 Python 编程语言编写的软件包，旨在设置、控制和分析原子模拟。在 ASE 中，任务完全在 Python 中进行脚本编写。Python 的强大语法与 NumPy 数组库相结合，使得执行非常复杂的模拟任务成为可能。例如，可以使用简单的“for循环”结构执行一系列计算。能量、力、应力和其他量的计算是通过使用统一接口与许多外部电子结构代码或力场进行接口操作完成的。在此计算器接口之上，ASE提供了许多标准模拟任务的模块，例如结构优化、分子动力学、处理约束并执行受激弹性带计算。

在本节中，我们将计算氮分子在铜表面上的吸附能。这是通过计算孤立的晶面和孤立分子的总能量来完成的。然后将吸附物添加到晶面上并进行松弛，计算此复合系统的总能量。吸附能量是通过将孤立能量之和减去复合系统的能量得到。

这是弛豫后的系统的图片：

![N2Cu](https://wiki.fysik.dtu.dk/ase/_images/surface.png)

让我们从构建原子开始吧！

#### 1.1 原子 <a id='1-1'></a>

[Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms) 对象是原子的集合。以下是如何通过直接指定两个氮原子的位置来定义 $N_2$ 分子的方法：

In [2]:
from ase import Atoms

d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

您也可以使用lattice模块构建晶体，该模块返回对应于常见晶体结构的 Atoms 对象。让我们制作一个Cu（111）表面：

In [3]:
from ase.build import fcc111

slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### 1.2 添加计算 <a id='1-2'></a>

在这个概述中，我们使用有效介质理论（EMT）计算器，因为它非常快速，因此非常适用于本案例。我们可以将计算器附加到先前创建的 Atoms 对象上：

In [4]:
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()

并使用Atoms类的get_potential_energy()方法来使用它计算系统的总能量：

In [5]:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()
print(f'N2 的能量是：{e_N2}')
print(f'Cu 的能量是：{e_slab}')

N2 的能量是：0.44034357303561467
Cu 的能量是：11.509056283570382


#### 1.3 结构弛豫（使结构优化到能量最低状态）<a id='1-3'></a>

让我们使用拟牛顿优化器优化 $N_2$ 分子在铜表面上吸附的结构。首先将吸附剂添加到铜片上，例如放置在上方位置：

In [6]:
from ase.build import add_adsorbate

h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

为了加快弛豫速度，让我们使用来自约束模块的 FixAtoms 将 Cu 原子固定在板中。然后只允许 $N_2$ 分子松弛到平衡结构：

In [7]:
from ase.constraints import FixAtoms

constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

现在将拟牛顿最小化器连接到系统并保存轨迹文件。使用收敛准则运行最小化器，即所有原子的力应小于某个 fmax：

In [8]:
from ase.optimize import QuasiNewton

dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 16:07:19       11.689927*       1.0797
BFGSLineSearch:    1[  2] 16:07:19       11.670814*       0.4090
BFGSLineSearch:    2[  4] 16:07:19       11.625880*       0.0409


True

到这里我们就得到了弛豫后体系的总能量是 11.803869 eV。

让我们计算一下吸附能。

In [9]:
print('吸附能:', e_slab + e_N2 - slab.get_potential_energy())

吸附能: 0.32351942231800734


#### 1.4 总结 <a id='1-4'></a>

让我们将以上各部分组成一个连续的代码：

In [10]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('吸附能:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 16:07:19       11.689927*       1.0797
BFGSLineSearch:    1[  2] 16:07:19       11.670814*       0.4090
BFGSLineSearch:    2[  4] 16:07:19       11.625880*       0.0409
吸附能: 0.32351942231800734


### 2 输入与输出 <a id='io'></a>

使用 write() 函数将原子位置写入文件：

In [11]:
from ase.io import write

write('slab.xyz', slab)

/opt/conda/lib/python3.8/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


这将以xyz格式编写文件。可能的格式有：

| 格式   | 描述                       |
| ------ | -------------------------- |
| `xyz`  | 简单的xyz格式              |
| `cube` | 高斯立方体文件             |
| `pdb`  | 蛋白质数据储存文件         |
| `traj` | ASE自己的轨迹格式          |
| `py`   | Python脚本                 |

从文件中读取的方法如下：

In [12]:
from ase.io import read

slab_from_file = read('slab.xyz')

### 3 可视化 <a id='visualize'></a>

可视化原子的最简单的方法是使用view()函数：

In [13]:
from ase.visualize import view

# view(slab)

view(slab) 将弹出一个 ase.gui 窗口。

但请注意，默认的 viewer 使用的是 ase.gui 窗口，无法在 notebook 中展示。

可以通过指定可选的关键字 viewer=... 来使用备用查看器。

（请注意，这些备用查看器不是ASE的一部分，用户需要单独安装。） 

在 bohrium notebook 中，最佳的 viewer 参数是 `ngl` 或 `x3d` 

`ngl` 需要 nglview 的支持。一般来说，你的镜像已经安装了nglview，如果没有安装，请执行以下命令

In [14]:
# view(slab, viewer='x3d')
view(slab, viewer='ngl')

### 4 运行分子动力学（Molecular dynamics）计算 <a id='molecular-dynamics'></a>

让我们以氮分子为例，用 VelocityVerlet 算法来看分子动力学。

我们首先创建 VelocityVerlet 对象，将其赋予分子和用于牛顿定律积分的时间步长。

然后，我们通过调用其 run() 方法并告诉它要执行多少步骤来执行动力学：

In [15]:
from ase.md.verlet import VelocityVerlet
from ase import units

dyn = VelocityVerlet(molecule, dt=1.0 * units.fs)

for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=20)

/opt/conda/lib/python3.8/site-packages/ase/md/verlet.py:42: FutureWarning: dt variable is deprecated; please use timestep.
  warnings.warn(FutureWarning('dt variable is deprecated; please use timestep.'))


 0: 0.44034 eV, 0.44034 eV, 0.00000 eV
 1: 0.43816 eV, 0.26289 eV, 0.17527 eV
 2: 0.44058 eV, 0.43142 eV, 0.00916 eV
 3: 0.43874 eV, 0.29292 eV, 0.14582 eV
 4: 0.44015 eV, 0.41839 eV, 0.02176 eV
 5: 0.43831 eV, 0.28902 eV, 0.14929 eV
 6: 0.43947 eV, 0.36902 eV, 0.07045 eV
 7: 0.43951 eV, 0.35507 eV, 0.08444 eV
 8: 0.43959 eV, 0.36221 eV, 0.07738 eV
 9: 0.43933 eV, 0.36044 eV, 0.07889 eV


## 总结 <a id='summary'></a>

In [16]:
# 运行完毕后删除生成的临时文件
import os

tmp_files = ['N2Cu.traj', 'slab.xyz']
for i in tmp_files:
    os.remove(i)

在本教程中，您学习了在 ASE 中的一些基础方法。 

具体而言，您了解到： 
- 使用 ASE 定义分子或晶体。
- 计算体系能量并进行结构弛豫。
- 使用 ASE 读取或写入原子文件
- 可视化原子文件
- 使用 ASE 进行分子动力学计算
 
你有什么问题吗？ 欢迎与我们联系 [bohrium@dp.tech](mailto:bohrium@dp.tech) 。

## 参考

1. https://wiki.fysik.dtu.dk/ase/gettingstarted/surface.html#
2. https://gitlab.com/ase/ase/-/tree/master

<a href="https://bohrium.dp.tech/notebook/4918f71cdc61483389e060abfb04d324" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>